In [12]:
# Functional -- Updated 2024.11.2
import requests
from bs4 import BeautifulSoup
import re
import os
from urllib.parse import urljoin, urlparse
from urllib.request import urlretrieve
from urllib.parse import unquote
import img2pdf


# The base URL for UFC results
base_url = "https://www.ufc.com"
results_url = base_url + "/results?language_content_entity=en"

def sanitize_filename(filename):
    """
    Cleans and sanitizes a filename by removing or replacing characters that may cause issues on various operating systems.
    This function also limits the filename length to 100 characters to ensure compatibility across different filesystems.

    Parameters:
    - filename (str): The original filename that may contain special or unsafe characters.

    Returns:
    - str: A sanitized filename with problematic characters replaced or removed, truncated to 100 characters.

    Process:
    1. Replace URL-encoded characters (e.g., `%20`) with underscores:
       - Uses a regular expression to match any URL-encoded sequences (% followed by two hexadecimal digits) 
         and replaces them with an underscore (`_`).
    2. Remove square brackets (`[`, `]`):
       - Removes any square brackets as they may cause issues on some systems.
    3. Replace special characters with underscores:
       - Replaces any characters not in the set `[A-Za-z0-9_.-]` with underscores to ensure compatibility across systems.
    4. Limit filename length:
       - Truncates the filename to a maximum of 100 characters to prevent issues with filesystems that have length limits.

    Example:
    - Input: "UFC%20266%20Volkanovski%20vs%20Ortega[43].jpg"
    - Output: "UFC_266_Volkanovski_vs_Ortega_43_.jpg"

    Notes:
    - This function is essential for ensuring that filenames are safe for use on various filesystems and
      avoids issues related to special characters, URL encoding, and excessive length.
    """
    filename = unquote(filename)  # Decode URL-encoded characters like %20, %5B, etc.
    filename = re.sub(r'[^\w\-_\.]', '_', filename)  # Replace special characters with underscores
    filename = filename.replace('.', '_')  # Replace periods that might interfere with file creation
    return filename[:50]  # Truncate filename to prevent path length issues

def extract_url_text_to_compare_previous_link(url):
    """
    Extracts a unique identifier from a UFC event URL by capturing the segment between 'ufc' and 'vs'.
    This function is designed to identify and track events based on a specific part of the URL,
    helping to detect redundant entries when multiple URLs refer to the same event.

    Parameters:
    - url (str): The URL of the UFC event page from which to extract the identifier.

    Returns:
    - str or None: A string representing the unique identifier extracted from the URL.
                   If the URL does not match the expected pattern, returns None and prints a message.

    Process:
    1. Use a regular expression to search for a segment in the URL that appears between 'ufc' and 'vs'.
       - The pattern 'ufc-(.*?)-vs' captures any text occurring between these keywords.
    2. If a match is found:
       - Return the captured portion as the unique identifier.
    3. If no match is found:
       - Print a message indicating that no unique identifier was found.
       - Return None.

    Example:
    - Input: "https://www.ufc.com/news/ufc-257-poirier-vs-mcgregor"
    - Output: "257-poirier"

    Notes:
    - This function helps in identifying and avoiding duplicate event entries by focusing on the URL structure.
    - It is particularly useful when URLs vary slightly but refer to the same event, allowing efficient comparison.
    """
    match = re.search(r'ufc-(.*?)-vs', url)
    if match:
        return match.group(1)
    
    print("No unique identifier found")
    return None

def extract_unique_identifier(url):
    """
    Extracts a unique identifier from a UFC event URL to help identify and differentiate event pages.
    This function focuses on capturing the part of the URL that follows '/news/', which typically includes
    event-specific information and is used to prevent duplicate entries.

    Parameters:
    - url (str): The URL of the UFC event page from which to extract a unique identifier.

    Returns:
    - str or None: A string representing the unique identifier extracted from the URL.
                   If the URL does not match the expected pattern, returns None and prints a message.

    Process:
    1. Use a regular expression to search for the part of the URL following '/news/'.
       - The pattern captures everything after '/news/' up to the end of the string.
    2. If a match is found:
       - Return the captured portion as the unique identifier.
    3. If no match is found:
       - Print a message indicating that no unique identifier was found.
       - Return None.

    Example:
    - Input: "https://www.ufc.com/news/ufc-257-official-scorecards"
    - Output: "ufc-257-official-scorecards"

    Notes:
    - This function is essential for identifying and tracking unique events, especially when URLs follow
      different structures. By extracting the event-specific portion, it enables better data organization.
    """
    match = re.search(r'/news/(.*)', url)
    if match:
        return match.group(1)
    
    print("No unique identifier found")
    return None

def find_results_links(url):
    """
    Navigates through the UFC results pages to gather links to individual fight result pages.
    This function filters out unnecessary links (e.g., weigh-ins or bonus announcements) and removes
    redundant entries by comparing unique identifiers for each event. If a "Load more" button is present,
    the function paginates to collect links from additional pages.

    Parameters:
    - url (str): The starting URL of the UFC results page to begin scraping fight result links.

    Returns:
    - list: A list of URLs (strings) linking to individual fight result pages for unique events.
    
    Process:
    1. Initialize:
       - `results_links` to store collected result links.
       - `last_fight_info` to track the most recent unique event identifier, used to prevent redundant entries.
    2. While there are more pages (indicated by the `url` variable):
       - Print the current URL for debugging purposes.
       - Request the page content and parse it using BeautifulSoup.
       - Find all `<a>` tags with class "c-card--grid-card-trending" which represent event result links.
    3. For each link found:
       - Convert the link's text to lowercase and extract its `href` attribute.
       - Filter links that include "results" but exclude keywords like "weigh," "bonus," "episode," and "season."
       - Use `extract_url_text_to_compare_previous_link` to retrieve a unique identifier for the event.
       - If the identifier (`current_fight_info`) differs from `last_fight_info`, add the link to `results_links`.
       - Update `last_fight_info` with `current_fight_info` to avoid adding duplicates.
    4. Check for pagination:
       - If a "Load more items" button is found, extract its `href` attribute.
       - Adjust `url` to point to the next page, accounting for both relative and absolute URLs.
       - If no "Load more" button is found, set `url` to `None` to end pagination.
    5. Return `results_links` after collecting links from all pages.

    Example:
    - Input: The URL of the UFC results page (e.g., "https://www.ufc.com/results?language_content_entity=en").
    - Output: A list of URLs linking to unique fight result pages, filtering out irrelevant entries.

    Notes:
    - This function enables comprehensive link collection across multiple pages while filtering out unnecessary links.
    - The pagination process ensures all available result links are gathered, supporting event tracking.
    """
    
    results_links = []  # Initialize a list to store collected results links
    last_fight_info = None  # Track last event identifier to prevent redundant entries
    
    while url:
        print("Processing:", url)  # Debug: show the current page URL being processed
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all relevant event result links on the page
        for link in soup.find_all("a", class_="c-card--grid-card-trending"):
            link_text = link.text.lower()
            link_href = link.get('href')
            # Filter by links containing 'results' but excluding weigh-ins, bonuses, etc.
            if "results" in link_text and "weigh" not in link_text and "bonus" not in link_text and "episode" not in link_text and "season" not in link_text:
                # Extract event identifier to prevent duplicate links for the same event
                current_fight_info = extract_url_text_to_compare_previous_link(link_href)
                if current_fight_info != last_fight_info:
                    full_link = base_url + link_href
                    results_links.append(full_link)
                    last_fight_info = current_fight_info  # Update last identifier for comparison
        
        # Check for 'Load more' button to proceed to next page of results
        load_more_button = soup.find("a", title="Load more items")
        if load_more_button:
            next_page = load_more_button.get('href')
            # Adjust for relative URL if present, otherwise proceed to the next absolute URL
            if next_page.startswith("?"):
                url = base_url + "/results" + next_page
            else:
                url = next_page
        else:
            url = None  # Stop if no more pages are available
            
    return results_links

def find_event_scorecard_links(results_links):
    """
    Processes each result link to identify official scorecard links associated with each UFC event.
    This function checks for duplicate events based on unique identifiers and retrieves scorecard links.
    For links that do not directly contain scorecard URLs, it parses the page to find embedded scorecard links.

    Parameters:
    - results_links (list): A list of URLs (strings) linking to UFC event result pages.

    Returns:
    - list: A list of URLs that point to official scorecards for each unique UFC event.
    
    Process:
    1. Initialize:
       - `event_scorecard_links` to store the final set of scorecard links.
       - `last_identifier` to track the most recent unique event identifier, preventing duplicate entries.
    2. Iterate over each `url` in `results_links`:
       - Extract a unique identifier (`current_identifier`) from `url` using `extract_unique_identifier`.
       - If `current_identifier` differs from `last_identifier`, process the link to avoid duplicates.
       - For URLs already containing 'official scorecard' or 'official-judges-scorecard':
         - Append the URL directly to `event_scorecard_links`.
         - Update `last_identifier` to `current_identifier` to mark this event as processed.
       - For other URLs:
         - Request the page content and parse it to locate any scorecard links embedded within the page.
         - Find links containing 'official' and 'scorecard' in the `href` attribute.
         - If a scorecard link is found, check if it’s a relative URL and adjust accordingly.
         - Extract a new identifier from the found scorecard link.
         - If the new identifier differs from `last_identifier`, add the link to `event_scorecard_links` and update `last_identifier`.
    3. Return the `event_scorecard_links` list, which contains URLs of unique scorecard links for each event.

    Example:
    - Input: A list of event result links, some containing scorecards directly and others requiring page parsing.
    - Output: A list of unique, official scorecard links for the events.

    Notes:
    - This function is essential for identifying unique scorecard links efficiently while avoiding duplicate entries.
    - It also handles both direct and indirect scorecard links by parsing each page if needed.
    """
    event_scorecard_links = []  # List to store the final set of scorecard links
    last_identifier = None  # Track last identifier to avoid duplicate scorecard entries
    
    for url in results_links:
        print("Processing:", url)  # Debug: show the URL currently being processed
        # Extract unique identifier to detect if the link represents a duplicate event
        current_identifier = extract_unique_identifier(url)
        if current_identifier != last_identifier:
            # If the URL already contains 'official scorecard', add it directly to the list
            if "official-judges-scorecard" in url or "official-scorecard" in url:
                print("result_link same as scorecard_link:", url)
                event_scorecard_links.append(url)
                last_identifier = current_identifier
            else:
                print("Visiting result_link:", url)  # Debug: opening link to look for scorecard
                response = requests.get(url)
                soup = BeautifulSoup(response.content, 'html.parser')

                # Find links containing 'official' and 'scorecard' within the page
                scorecard_link_element = soup.find("a", href=lambda href: href and "official" in href and "scorecard" in href)
                if scorecard_link_element:
                    scorecard_link = scorecard_link_element.get('href')
                    # Adjust for relative URLs if necessary
                    formatted_scorecard_link = scorecard_link if scorecard_link.startswith("http") else base_url + scorecard_link
                    
                    # Re-extract identifier for the updated URL structure, if different
                    new_identifier = extract_unique_identifier(formatted_scorecard_link)
                    if new_identifier != last_identifier:
                        event_scorecard_links.append(formatted_scorecard_link)
                        last_identifier = new_identifier
    
    return event_scorecard_links

def remove_redundant_links(event_scorecard_links):
    """
    Removes redundant links from a list of event scorecard links. This function filters out
    specific unwanted links and eliminates redundant links that are substrings of other links.
    
    Parameters:
    - event_scorecard_links (list): A list of URLs (strings) for UFC event scorecards that may contain redundant links.

    Returns:
    - tuple: A tuple containing two lists:
        1. `filtered_links` - A list of unique, non-redundant links.
        2. `removed_event_scorecard_links` - A list of links that were identified as redundant and removed.
    
    Process:
    1. Initialize:
       - `removed_event_scorecard_links` to store links that are filtered out or deemed redundant.
       - `filtered_links` to store the final set of unique links.
       - `links_to_remove` as a set for efficient identification of links to be removed.
    2. Filter out links containing 'search?query=official+scorecards' as they are considered unnecessary:
       - Add these links to `links_to_remove`.
    3. Check each link against all other links to detect redundancy:
       - For each link `link_i`, compare it with all other links `link_j`.
       - If `link_i` is a substring within `link_j`, mark `link_j` as redundant by adding it to `links_to_remove`.
    4. Separate links based on `links_to_remove`:
       - If a link is in `links_to_remove`, add it to `removed_event_scorecard_links`.
       - Otherwise, add it to `filtered_links`.
    5. Return the `filtered_links` (unique links) and `removed_event_scorecard_links` (redundant links).
    
    Example:
    - Input: ["link1", "link1-extra", "link2", "link2-extra", "link1"]
    - Output: 
      - `filtered_links`: ["link1", "link2"]
      - `removed_event_scorecard_links`: ["link1-extra", "link2-extra"]

    Notes:
    - This function helps streamline the list of event scorecard links by removing redundancies, 
      making the list more accurate and concise.
    """
    
    removed_event_scorecard_links = []  # List of links removed
    filtered_links = []  # List of links kept after filtering
    links_to_remove = set()  # Set for efficient removal

    # Filter out links containing 'search?query=official+scorecards'
    for link in event_scorecard_links:
        if 'search?query=official+scorecards' in link:
            links_to_remove.add(link)
    
    # Check each link against all others to detect redundancies
    for i, link_i in enumerate(event_scorecard_links):
        for j, link_j in enumerate(event_scorecard_links):
            # Avoid self-comparison
            if i == j:
                continue
            # Mark link_j for removal if link_i is contained within it
            if link_i in link_j and link_j not in links_to_remove:
                links_to_remove.add(link_j)
    
    # Separate filtered and removed links based on the removal set
    for link in event_scorecard_links:
        if link in links_to_remove:
            removed_event_scorecard_links.append(link)
        else:
            filtered_links.append(link)
    
    return filtered_links, removed_event_scorecard_links

def add_manual_event_scorecard_links(existing_links, manual_links):
    """
    Adds manually identified scorecard links to an existing list of links, ensuring no duplicates.
    This function is useful when some event scorecard links are missed by automated scraping, allowing 
    them to be added manually.
    
    Parameters:
    - existing_links (list): The list of event scorecard links gathered through automated scraping.
    - manual_links (list): The list of additional event scorecard links identified manually.

    Returns:
    - list: A combined list of event scorecard links with duplicates removed.
    
    Process:
    1. Convert `existing_links` to a set (`existing_set`) to enable quick lookup of links and avoid duplicates.
    2. Initialize `combined_links` with the `existing_links` list to start with all previously collected links.
    3. Iterate over each link in `manual_links`:
       - Check if the link is already in `existing_set` (to avoid redundancy).
       - If the link is not present, add it to `combined_links` and update `existing_set`.
    4. Return the `combined_links` list, which now includes all unique links from both `existing_links` and `manual_links`.

    Example:
    - If `existing_links` = ["link1", "link2"] and `manual_links` = ["link2", "link3"],
      the returned list will be ["link1", "link2", "link3"], with "link2" appearing only once.

    Notes:
    - This function is designed to prevent duplicates efficiently by using a set for lookups.
    - It’s particularly useful when certain links are missed by automated scraping and need to be appended manually.
    """
    existing_set = set(existing_links)  # Set for quick lookup of existing links
    combined_links = list(existing_links)  # Initialize with existing links
    for link in manual_links:
        # Only add the link if it does not already exist in the set
        if link not in existing_set:
            combined_links.append(link)
            existing_set.add(link)
    return combined_links

# Manual links identified that were missed by the scraping process
manual_event_scorecard_links = [
    "https://www.ufc.com/news/official-judges-scorecards-ufc-fight-night-ankalaev-vs-walker-2",
    "https://www.ufc.com/news/official-scorecards-ufc-fight-night-ortega-vs-rodriguez",
    "https://www.ufc.com/news/official-judges-scorecards-ufc-269-oliveira-vs-poirier-nunes-pena?language_content_entity=en",
    "https://www.ufc.com/news/official-scorecards-ufc-fight-night-sandhagen-vs-dillashaw",
    "https://www.ufc.com/news/ufc-fight-night-brunson-vs-holland-scorecards-gillespie-riddell-winners-ufc-vegas-22?language_content_entity=en",
    "https://www.ufc.com/news/ufc-257-official-scorecards-poirier-mcgregor-2-fight-island",
    "https://www.ufc.com/news/ufc-fight-island-6-official-scorecards-ortega-korean-zombie-results",
    "https://www.ufc.com/news/ufc-vegas-9-official-scorecards-sakai-overeem",
    "https://www.ufc.com/news/ufc-vegas-8-official-scorecards-fight-night-smith-rakic?language_content_entity=en",
    "https://www.ufc.com/news/ufc-vegas-7-official-scorecards-munhoz-edgar",
]


def parse_folder_name_from_event_scorecard_link(event_scorecard_link):
    """
    Parses a folder name from the event scorecard link by extracting a specific part of the URL.
    The function aims to create a folder name that is compatible with file system requirements.
    
    Parameters:
    - event_scorecard_link (str): The URL of the event scorecard, typically containing a segment 
      that uniquely identifies the event (e.g., "/news/ufc-257-official-scorecards").
    
    Returns:
    - str: A sanitized folder name based on the URL segment following 'news/'.
           If the expected URL pattern is not matched, returns "No_Folder_Name_Parsed".
    
    Process:
    1. Use a regular expression to extract the portion of the URL following 'news/' up to a query parameter 
       ('?'), the string "results," or the end of the URL.
    2. Replace hyphens ('-') in the extracted segment with underscores ('_') to ensure compatibility with file systems.
    3. Remove or replace characters that are invalid in file or folder names (such as \, /, *, ?, :, ", <, >, |).
    4. Return the sanitized folder name. If no match is found in the URL, return "No_Folder_Name_Parsed".
    
    Example:
    - Input: "https://www.ufc.com/news/ufc-257-official-scorecards?language_content_entity=en"
    - Output: "ufc_257_official_scorecards"
    
    Notes:
    - This function is essential for organizing downloaded images and PDFs into folders 
      named after the event, ensuring that folder names are safe for file storage.
    """
    match = re.search(r'news/(.*?)(\?|results|$)', event_scorecard_link)
    if match:
        folder_name = match.group(1).replace('-', '_').strip()
        folder_name = re.sub(r'[\\/*?:"<>|]', '', folder_name)
        return folder_name[:30]  # Shorten folder name to prevent path issues
    else:
        return "No_Folder_Name_Parsed"


def download_images_from_event_scorecard_link_and_create_pdf(event_scorecard_link, event_index):
    """
    Downloads all images from the specified event scorecard link and compiles them into a PDF.
    
    Parameters:
    - event_scorecard_link (str): URL of the event scorecard page containing images to download.
    - event_index (int): A unique identifier or index for the event, used in naming directories and files.
    
    Process:
    1. Parse a folder name based on the event scorecard link to organize files.
       - Calls `parse_folder_name_from_event_scorecard_link` to ensure the folder name is file system-friendly.
    2. Create a directory within "ufc_images" to store images and the PDF for this event.
    3. Send an HTTP request to the event scorecard link and parse the page content using BeautifulSoup.
    4. Locate all image elements on the page with class "image-style-inline".
       - Filter images to include only those hosted on "dmxg5wxfqgb4u.cloudfront.net" for consistency.
    5. For each filtered image:
       - Extract the image URL and sanitize the filename using `sanitize_filename` to ensure compatibility.
       - Download each image to the specified directory.
       - Append each downloaded image's path to `image_paths` for PDF creation.
    6. If any images were successfully downloaded, compile them into a single PDF:
       - Name the PDF file with the format "{event_index}_{parsed_folder_name}.pdf".
       - Save the PDF in the event's directory.
       - Print a success message; if PDF creation fails, print an error message.

    Output:
    - Downloads images from the event scorecard link into the event’s directory.
    - Creates a PDF of all downloaded images in the event’s directory if downloads are successful.
    - Prints progress and error messages to help track each step of the process.
    """
    
    parsed_folder_name = parse_folder_name_from_event_scorecard_link(event_scorecard_link)[:30]
    save_dir = os.path.join("ufc_images", f"event_{event_index}_{parsed_folder_name}")
    os.makedirs(save_dir, exist_ok=True)  # Ensure directory exists

    response = requests.get(event_scorecard_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all unique image URLs
    img_tags = soup.find_all('img', class_="image-style-inline")
    unique_images = set(img['src'] for img in img_tags if "dmxg5wxfqgb4u.cloudfront.net" in img.get('src', ''))

    image_paths = []  # List to store paths of downloaded images

    # Download each unique image
    for count, img_src in enumerate(unique_images):
        img_url = urljoin(event_scorecard_link, img_src)
        original_filename = os.path.basename(urlparse(img_url).path)
        clean_filename = sanitize_filename(original_filename)
        
        # Ensure filename ends with .jpg
        if not clean_filename.lower().endswith('.jpg'):
            clean_filename += '.jpg'
        
        # Add a sequence number to ensure uniqueness
        filename = os.path.join(save_dir, f"{count}_{clean_filename}")

        try:
            os.makedirs(save_dir, exist_ok=True)  # Double-check directory creation
            
            # Fetch the image content
            img_response = requests.get(img_url)
            if img_response.headers['Content-Type'].startswith('image'):
                with open(filename, 'wb') as f:
                    f.write(img_response.content)
                print(f"Downloaded {filename}")
                image_paths.append(filename)
            else:
                print(f"Skipped non-image content at {img_url}")

        except Exception as e:
            print(f"Could not download {img_url}. Reason: {e}")

    # If images were downloaded, compile them into a PDF
    if image_paths:
        pdf_filename = os.path.join(save_dir, f"{event_index}_{parsed_folder_name}.pdf")
        try:
            with open(pdf_filename, "wb") as f:
                f.write(img2pdf.convert(image_paths))
            print(f"Created PDF: {pdf_filename}")
        except Exception as e:
            print(f"Failed to create PDF: {pdf_filename}. Error: {e}")

def image_processing(event_scorecard_links, starting_event_number=1, ending_event_number=None):
    """
    Processes images and creates PDFs for UFC event scorecards within a specified range of events.
    
    Parameters:
    - event_scorecard_links (list): A list of URLs (strings) for UFC event scorecards to process.
    - starting_event_number (int): The first event number to process in the list (default is 1).
    - ending_event_number (int or None): The last event number to process. If None, processes to the end of the list.
    
    Process:
    1. Determine the range of event scorecard links to process.
        - If `ending_event_number` is not specified, set it to cover the entire `event_scorecard_links` list.
    2. Loop through the specified range of `event_scorecard_links`, starting from `starting_event_number`.
        - For each event in the specified range, output a message indicating the event URL being processed.
        - Call `download_images_from_event_scorecard_link_and_create_pdf` with each scorecard link and its index as arguments.
          This function downloads images from the scorecard link and compiles them into a PDF.

    Output:
    - Downloads and compiles images into a PDF for each specified event scorecard.
    - Prints progress messages for each event processed, aiding in tracking the processing flow.
    
    Example:
    - To process events 1 through 10:
      `image_processing(event_scorecard_links, 1, 10)`
    - To process all events from event 5 onward:
      `image_processing(event_scorecard_links, 5)`
    """
    
    # Set ending_event_number to the length of event_scorecard_links if it is None
    if ending_event_number is None:
        ending_event_number = len(event_scorecard_links)

    # Loop through the specified range of event scorecard links
    for index, event_scorecard_link in enumerate(event_scorecard_links[starting_event_number - 1:ending_event_number], start=starting_event_number):
        print(f"Processing images from: {event_scorecard_link}")
        download_images_from_event_scorecard_link_and_create_pdf(event_scorecard_link, index)
        
        
# PDF Checker and Creator Functions

def find_folders_without_pdfs(parent_directory):
    """
    Finds and returns a list of folder paths within the specified parent directory
    that do not contain any PDF files.
    
    Parameters:
    - parent_directory (str): The path to the directory to search within for folders lacking PDFs.
    
    Returns:
    - List[str]: A list of folder paths that do not contain any PDF files.
    
    Process:
    1. Walk through all directories and files in the specified parent directory.
    2. For each directory, check if any files with a '.pdf' extension are present.
    3. If a directory does not contain a PDF file, add its path to the 'folders_without_pdfs' list.
    4. Return the list of directories without PDFs.
    """
    folders_without_pdfs = []
    
    # Walk through all directories and files in the specified parent directory
    for root, dirs, files in os.walk(parent_directory):
        # Convert file names to lowercase for case-insensitive comparison
        files_lower = [file.lower() for file in files]
        
        # Check if there are no files at all or no PDF files (case-insensitive) in the current directory
        if not files or not any(file.endswith('.pdf') for file in files_lower):
            folders_without_pdfs.append(root)  # Add directory path to the list if no PDF is found
            
    return folders_without_pdfs

def create_pdf_from_images(folder_path):
    """
    Creates a PDF from all JPEG images in the specified folder.
    
    Parameters:
    - folder_path (str): The path to the folder containing JPEG images to convert into a PDF.
    
    Process:
    1. Collect all JPEG (.jpg or .jpeg) image files in the specified folder and sort them.
    2. Define a PDF filename ("compiled_images.pdf") within the folder path.
    3. If JPEG images are found:
        a. Convert each image file path into a PDF format using the img2pdf library.
        b. Write the resulting PDF to the specified filename.
    4. If no JPEG images are found, display a message indicating this.
    
    Output:
    - Creates "compiled_images.pdf" in the folder if JPEG images are present.
    - Prints success or error messages based on the process outcome.
    """
    image_files = sorted(
        [f for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.jpeg')]
    )
    pdf_filename = os.path.join(folder_path, "compiled_images.pdf")
    if image_files:
        image_paths = [os.path.join(folder_path, f) for f in image_files]
        try:
            with open(pdf_filename, "wb") as f:
                f.write(img2pdf.convert(image_paths))
            print(f"PDF created successfully: {pdf_filename}")
        except Exception as e:
            print(f"Failed to create PDF from images in {folder_path}. Error: {e}")
    else:
        print(f"No JPEG images found in {folder_path}.")


# Main Execution Flow
def main():
    # Collect result links and process them
    results_links = find_results_links(results_url)
    event_scorecard_links = find_event_scorecard_links(results_links)
    event_scorecard_links, removed_event_scorecard_links = remove_redundant_links(event_scorecard_links)
    event_scorecard_links = add_manual_event_scorecard_links(event_scorecard_links, manual_event_scorecard_links)
    
    # Print diagnostic information
    print("\nFound event scorecard links:")
    for link in event_scorecard_links:
        print(link)
    
    # Process images and create PDFs for the full dataset by default
    image_processing(event_scorecard_links)

    # Check for folders without PDFs
    parent_directory = r"C:\Users\EditZ\ufc_images"
    folders_without_pdfs = find_folders_without_pdfs(parent_directory)

    print("\nFolders without PDF files:")
    for folder in folders_without_pdfs:
        print(folder)

    # Attempt to create PDFs for folders without any existing PDFs
    for folder in folders_without_pdfs:
        print(f"Processing folder for PDF creation: {folder}")
        create_pdf_from_images(folder)

if __name__ == "__main__":
    main()

Processing: https://www.ufc.com/results?language_content_entity=en
No unique identifier found
Processing: https://www.ufc.com/results?language_content_entity=en&page=1
No unique identifier found
No unique identifier found
Processing: https://www.ufc.com/results?language_content_entity=en&page=2
No unique identifier found
No unique identifier found
No unique identifier found
Processing: https://www.ufc.com/results?language_content_entity=en&page=3
No unique identifier found
No unique identifier found
No unique identifier found
Processing: https://www.ufc.com/results?language_content_entity=en&page=4
No unique identifier found
Processing: https://www.ufc.com/results?language_content_entity=en&page=5
No unique identifier found
Processing: https://www.ufc.com/results?language_content_entity=en&page=6
No unique identifier found
No unique identifier found
Processing: https://www.ufc.com/results?language_content_entity=en&page=7
No unique identifier found
No unique identifier found
No unique 

Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.


Downloaded ufc_images\event_1_official_judges_scorecards_ufc\12_Screenshot_2024-10-26_at_9_32_36_AM_png.jpg


Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.
Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.


Created PDF: ufc_images\event_1_official_judges_scorecards_ufc\1_official_judges_scorecards_ufc.pdf
Processing images from: https://www.ufc.com/news/official-judges-scorecards-ufc-fight-night-hernandez-vs-pereira
Downloaded ufc_images\event_2_official_judges_scorecards_ufc\0_101924-edwards-vidal-vegas-99-scorecard_jpg.jpg
Downloaded ufc_images\event_2_official_judges_scorecards_ufc\1_101924-hadley-smotherman-ufc-vegas-99-scorecard_jp.jpg
Downloaded ufc_images\event_2_official_judges_scorecards_ufc\2_101924-katona-matsumoto-ufc-vegas-99-scorecard_jpg.jpg
Downloaded ufc_images\event_2_official_judges_scorecards_ufc\3_101924-penne-reed-scorecard-ufc-vegas-99_jpg.jpg
Downloaded ufc_images\event_2_official_judges_scorecards_ufc\4_101924-martinez-ardelean-ufc-vegas-99-scorecard_jp.jpg
Downloaded ufc_images\event_2_official_judges_scorecards_ufc\5_101924-johnson-sumudaerji-ufc-vegas-99-scorecard_j.jpg
Downloaded ufc_images\event_2_official_judges_scorecards_ufc\6_101924-elkins-pineda-ufc-vega

...

Created PDF: ufc_images\event_90_official_scorecards_ufc_279_di\90_official_scorecards_ufc_279_di.pdf
Processing images from: https://www.ufc.com/news/official-judges-scorecards-ufc-fight-night-gane-vs-tuivasa-paris
Downloaded ufc_images\event_91_official_judges_scorecards_ufc\0_09032022-Scorecards_-_Imavov_vs__Buckley_jpeg.jpg
Downloaded ufc_images\event_91_official_judges_scorecards_ufc\1_09032022-Scorecard-Gomis-vs-Errens_jpeg.jpg
Downloaded ufc_images\event_91_official_judges_scorecards_ufc\2_09032022-Scorecards-Whittaker-vs-Vettori_jpeg.jpg
Downloaded ufc_images\event_91_official_judges_scorecards_ufc\3_09032022-Scorecards-Taha-vs-Quinonez_jpeg.jpg
Downloaded ufc_images\event_91_official_judges_scorecards_ufc\4_09032022-Scorecards-Di-Chirico-vs-Kopylov_jpeg.jpg
Downloaded ufc_images\event_91_official_judges_scorecards_ufc\5_09032022-Scorecards-Saint-Denis-vs-Miranda_jpeg.jpg
Downloaded ufc_images\event_91_official_judges_scorecards_ufc\6_09042022-Scorecards-Gane-vs-Tuivasa_jpeg.jp

Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.


Downloaded ufc_images\event_93_official_scorecards_ufc_fight_\12_Official-Scorecard-Angela-Hill-vs-Loopy-Godinez_jp.jpg
Created PDF: ufc_images\event_93_official_scorecards_ufc_fight_\93_official_scorecards_ufc_fight_.pdf
Processing images from: https://www.ufc.com/news/official-judges-scorecards-ufc-fight-night-santos-vs-hill-apex
Downloaded ufc_images\event_94_official_judges_scorecards_ufc\0_080622-ufc-fight-night-santos-vs-hill-cory-mckenna.jpg
Downloaded ufc_images\event_94_official_judges_scorecards_ufc\1_080622-ufc-fight-night-santos-vs-hill-augusto-saka.jpg
Downloaded ufc_images\event_94_official_judges_scorecards_ufc\2_080622-ufc-fight-night-santos-vs-hill-terrance-mck.jpg
Downloaded ufc_images\event_94_official_judges_scorecards_ufc\3_080622-ufc-fight-night-santos-vs-hill-bryan-battle.jpg
Downloaded ufc_images\event_94_official_judges_scorecards_ufc\4_080622-ufc-fight-night-santos-vs-hill-mayra-bueno-.jpg
Downloaded ufc_images\event_94_official_judges_scorecards_ufc\5_080622-

Image contains an alpha channel. Computing a separate soft mask (/SMask) image to store transparency in PDF.


Downloaded ufc_images\event_131_ufc_fight_night_smith_vs_spann\13_091821-UFC-FIGHT-NIGHT-ANTHONY-SMITH-VS-RYAN-SPANN.jpg
Created PDF: ufc_images\event_131_ufc_fight_night_smith_vs_spann\131_ufc_fight_night_smith_vs_spann.pdf
Processing images from: https://www.ufc.com/news/ufc-fight-night-brunson-vs-till-official-judges-scorecards-ufc-vegas-36
Downloaded ufc_images\event_132_ufc_fight_night_brunson_vs_til\0_UFC_Fight_Night_Brunson_vs__Till_-_Scorecards_-_Mc.jpg
Downloaded ufc_images\event_132_ufc_fight_night_brunson_vs_til\1_UFC_Fight_Night_Brunson_vs__Till_-_Scorecards_-_Sh.jpg
Downloaded ufc_images\event_132_ufc_fight_night_brunson_vs_til\2_UFC_Fight_Night_Brunson_vs__Till_-_Scorecards_-_Lu.jpg
Downloaded ufc_images\event_132_ufc_fight_night_brunson_vs_til\3_UFC_Fight_Night_Brunson_vs__Till_-_Scorecards_-_As.jpg
Downloaded ufc_images\event_132_ufc_fight_night_brunson_vs_til\4_UFC_Fight_Night_Brunson_vs__Till_-_Scorecards_-_Er.jpg
Downloaded ufc_images\event_132_ufc_fight_night_brunso